<center>
<img src="logo.png" height="900"> 
</center>


#  НесмУщённый и состоятельный записыватель курсов

__Это задание является дополнительным и необязательным для решения.__ 

Филипп записывает онлайн-курс по статистике. Он ориентируется на то, что каждая неделя курса должна идти около $100$ минут. В голове у Филиппа есть оцениватель времени. Он оценивает по длительности каждое видео и записыват свои прикидки на бумагу. Когда видео снято, Филипп смотрит как долго оно длилось и записывает итоговый результат на бумагу. В таблице `filipp_course.tsv` представлены записи Филиппа за $11$ отснятых недель. 

__Описание колонок:__

- `real_week` - хронологический номер недели
- `week` - какой по порядку записывалась неделя
- `type` - тип видео (доска, презентация, код)
- `estimate_time` - во сколько минут Филипп оценил время видео при разработке материала
- `real_time` - сколько реально длится это видео


In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("filipp_course.tsv", sep='\t')
print(df.shape)
df.head()

(129, 5)


real_week  week          type  estimate_time  real_time
0          6     1    blackboard              6       11.0
1          6     1    blackboard             10        7.0
2          6     1    blackboard             10       11.5
3          6     1    blackboard              5        6.0
4          6     1  presentation              5        3.0

Будем считать, что на протяжении всей записи у Филиппа в голове был один и тот же оцениватель (estimator). По этому оценивателю Филипп делал точечные оценки для каждого видео независимо друг от друга. Давайте поисследуем оцениватель Филиппа на различные статистические свойства.

__а)__ Посчитайте разницу между `estimate_time` и `real_time`. Запишите её в новую колонку `bias`. Это то, насколько каждая оценка Филиппа оказалась смещена. Посчитайте среднее смещение, запишите его в переменную `bias_mean`. Проинтерпретируйте получившееся число. Правда ли, что Филипп делает несмещённые оценки?

In [2]:
df['bias'] = df['estimate_time'] - df['real_time']
bias_mean = df['bias'].mean()
print(bias_mean)

-2.1790697674418604


In [3]:
# проверка того, что задание решено верно (задание необязательно - все тесты открыты)
assert np.abs(bias_mean  + 2.179) < 1e-2


__б)__ Посчитайте среднее смещение, которое делает Филипп для каждого типа контента. Для какого типа смещение оказывается самым низким? Запишите его в переменную `bias_min`. 

In [4]:
bias_min = df.groupby('type')['bias'].mean().min()
print(bias_min)

-2.8862745098039215


In [5]:
# проверка того, что задание решено верно
assert bias_min  + 1.47 < 1e-2


__в)__ Посмотрим как оцениватель Филиппа ведёт себя в динамике. Посчитайте среднее смещение для каждой недели. Как оно ведёт себя в динамике? Правда ли, что оно постепенно сходится к нулю? Чем поведение смещения в динамике отличается от асимптотической несмещённости? 

In [6]:
df.groupby('week')['bias'].mean()

week
1    -1.950000
2    -1.558333
3    -1.727273
4    -1.300000
5    -3.150000
6    -3.328571
7    -3.000000
8    -1.000000
9    -2.200000
10   -2.272727
11   -2.200000
Name: bias, dtype: float64

__г)__ Несмещённость - это свойство при фиксированном $n$. Состоятельность - свойство при $n \to \infty$. Представим себе, что Филипп переосмыслил свою стратегию оценки времени, которое он тратит на запись видео. Теперь он хочет оценивать время на запись следующего видео как среднее, посчитанное по всем видео, записанным ранее.

_Например:_ Филипп записал три видео: 

- презентация 4 минуты
- презентация 1 минута
- доска 4 минуты

Тогда он оценить веремя на следующее видео как $\frac{4 + 1 + 4}{3} = 3$. Создайте в табличке колонку `new_estimate` с такми оценками. Время на первое видео он оценивает как $6$ минут. Посмотрите как себя ведёт такая оценка в динамике. Как думаете, к чему она сходится по вероятности? Какой физический смысл стоит за такой оценкой? 

In [8]:
df['new_estimate'] = 0
df.loc[0, 'new_estimate'] = 6
for i in range(1, len(df)):
    df.loc[i, 'new_estimate'] = df['real_time'].iloc[:i].mean()
df[['real_week', 'week', 'new_estimate']]

C:\Users\ziiat\AppData\Local\Temp\ipykernel_5576\2452063949.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '9.833333333333334' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'new_estimate'] = df['real_time'].iloc[:i].mean()


real_week  week  new_estimate
0            6     1      6.000000
1            6     1     11.000000
2            6     1      9.000000
3            6     1      9.833333
4            6     1      8.875000
..         ...   ...           ...
124          4    11     10.448387
125          4    11     10.468800
126          4    11     10.480952
127          4    11     10.508661
128          4    11     10.543750

[129 rows x 3 columns]

__д)__ Посчитайте для каждого видео смещение. По аналогии с пунктом __а)__ найдите среднее смещение. Запишите его в переменную `bias_mean_new`. Стало ли оно меньше благодаря новому оценивателю? Почему? 

In [9]:
bias_mean_new = (df['new_estimate'] - df['real_time']).mean()
bias_mean_new

-0.6094253748224452

In [10]:
# проверка того, что задание решено верно
assert np.abs(bias_mean_new  + 0.609) < 1e-2
